# 02_anchor_generator
**This file used to generate anchors for UECFOOD100 dataset through kmeans.
Distance here is not Euclidean distance but IoU (Intersection over Union) in
accordance with original yolo-v2 paper.**

In [4]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

**k-means clustering**:  
- Input: k, set of points [w1,h1], [w2,h2], [w3, h3], ..., [wn, hn] 
- Place centroids c1, ..., ck at random locations (randomly select k [w,h] among input img wh)
- Repeat until convergence:
    - for each point [wi,hi]:
        - find nearest centroid cj. (np.argmin(new_distances, axis=1))
        - assign the point [wi,hi] to cluster j.
    - for each cluster j=1, ...,k: (update centroids)
        - new centroid cj=mean of all points [wi,hi] assigned to cluster j in previous step
        $\frac{1}{n_j}\sum\limits_{x_i\rightarrow c_j}x_i(a)$  (a means a particular attribute in this case a is wh[i])
- Stop when none of the cluster assignments change

In [5]:
def kmeans(wh, centroids, anchor_txt):
    num = wh.shape[0]  # total number of different wh pairs
    k, dim = centroids.shape
    iter = 0
    old_distances = np.zeros((num, k))
    _assignments = -np.ones(num)

    # iterate until
    while True:
        new_distances = []
        iter += 1
        for i in range(num):
            distance = 1 - IoU(wh[i], centroids)  # high IoU represents low distance
            new_distances.append(distance)
        new_distances = np.array(new_distances)
        print('Iter {}: distances: {}'.format(iter, np.sum((np.abs(old_distances - new_distances)))))

        # for each input img assign a centroid (select the closed one)
        assignments = np.argmin(new_distances, axis=1)
        if (assignments == _assignments).all():
            print('final centroids =', centroids)
            save_anchors(centroids, anchor_txt, wh_in_yolov2)
            return centroids
        else:
            centroid_sums = np.zeros((k, dim), np.float)
            for i in range(num):
                centroid_sums[assignments[i]] += wh[i]  # sum up attribute
            for j in range(k):
                # new centroids
                centroids[j] = centroid_sums[j] / np.sum(assignments == j)

            _assignments = assignments.copy()
            old_distances = new_distances.copy()

In [6]:
def save_anchors(centroids, anchor_txt, wh_in_yolov2):
    width_in_yolov2 = wh_in_yolov2[0]
    height_in_yolov2 = wh_in_yolov2[1]
    with open(anchor_txt, 'w') as file:
        anchors = centroids.copy()
        for i in range(anchors.shape[0]):
            anchors[i][0] *= width_in_yolov2 / 32.
            anchors[i][1] *= height_in_yolov2 / 32.
        widths = anchors[:, 0]
        sorted_indices = np.argsort(widths)  # return the indices that sort tht array
        print('anchors = ', anchors[sorted_indices])

        for i in sorted_indices:
            file.write('%0.2f, %0.2f\n' % (anchors[i, 0], anchors[i, 1]))

In [7]:
def avgIoU(wh, centroids):
    sum = 0.
    for i in range(wh.shape[0]):
        sum += max(IoU(wh[i], centroids))
    return sum / wh.shape[0]

**IoU - Intersection over Union **

In [8]:
def IoU(whi, centroids):
    """ Calculate IoU between current centroids with one in wh array to check if current
    centroids are suitable enough
    :param whi:
    :param centroids:
    :return:
    """
    IOU = []
    for centroid in centroids:
        c_w, c_h = centroid
        w, h = whi
        if c_w >= w and c_h >= h:
            iou = w * h / (c_w * c_h)
        elif c_w >= w and c_h <= h:
            iou = w * c_h / (w * h + (c_w - w) * c_h)
        elif c_w <= w and c_h >= h:
            iou = c_w * h / (w * h + (c_h - h) * c_w)
        else:
            iou = c_w * c_h / (w * h)
        IOU.append(iou)
    return np.array(IOU)

In [9]:
def coordinate2wh(coordinates, uec100_dims):
    coordinates = list(map(float, coordinates))
    w = (coordinates[2] - coordinates[0]) / uec100_dims[0]  # x2-x1
    h = (coordinates[3] - coordinates[1]) / uec100_dims[0]  # y2-y1
    return w, h

In [10]:
def gen_anchors(n_clusters, uec100_dims):
    dataset_disk = '/Volumes/JS/UECFOOD100_JS/'
    output_path = dataset_disk + 'generated_anchors'
    train_uec100 = dataset_disk + 'train_uec100.txt'

    if not os.path.exists(output_path):
        os.mkdir(output_path)

    wh = []

    with open(train_uec100, 'r') as file:
        for i, line in enumerate(file):
            if i > 0:
                line = line.rstrip('\n')
                line = line.split(' ')
                coordinates = line[2:]
                w, h = coordinate2wh(coordinates, uec100_dims)
                wh.append([w, h])
        wh = np.array(wh)

        if n_clusters == 0:  # make from 1 to 10 clusters and pick the best one
            avgIou = []
            for n_cluster in range(1, 11):
                anchor_txt = os.path.join(output_path, 'anchors_%d.txt' % (n_cluster))
                # randomly select n_cluster anchors from wh array which contain w,h for each img
                indices = [random.randrange(wh.shape[0]) for i in range(n_cluster)]
                centroids = wh[indices]
                centroids = kmeans(wh, centroids, anchor_txt)
                avgIou.append([n_cluster, avgIoU(wh, centroids)])
            avgIou = np.array(avgIou)
            plt.plot(avgIou[:, 0], avgIou[:, 1])
            plt.scatter(avgIou[:, 0], avgIou[:, 1], c='r')
            plt.xlabel('number of cluster')
            plt.ylabel('average IoU')
            plt.savefig('avg_iou')
            plt.show()
        else:
            anchor_txt = os.path.join(output_path, 'anchors_%d.txt' % (n_clusters))
            # randomly select n_cluster anchors from wh array which contain w,h for each img
            indices = [random.randrange(wh.shape[0]) for i in range(n_clusters)]
            centroids = wh[indices]
            kmeans(wh, centroids, anchor_txt)

        print('Done!')

In [11]:
wh_in_yolov2 = [416, 416]
uec100_dims = [800, 600]  # dataset image width=800, height=600
n_clusters = 0

In [ ]:
gen_anchors(n_clusters, uec100_dims)

Iter 1: distances: 4293.581308319727
Iter 2: distances: 893.1097914593079
final centroids = [[0.74009049 0.54857129]]
anchors =  [[9.62117636 7.13142682]]
Iter 1: distances: 7537.5998058491905
Iter 2: distances: 2259.651174610529
Iter 3: distances: 1448.0313179846505
Iter 4: distances: 703.145500303793
Iter 5: distances: 415.69694939175423
Iter 6: distances: 240.3678818628788
Iter 7: distances: 139.85304065898293
Iter 8: distances: 106.40824344115265
Iter 9: distances: 55.018533953462615
Iter 10: distances: 29.091589980444805
Iter 11: distances: 17.352063338900887
Iter 12: distances: 3.754916432679992
Iter 13: distances: 8.770189724026233
Iter 14: distances: 5.74932787966096
final centroids = [[0.86091727 0.63279566]
 [0.45535688 0.35009288]]
anchors =  [[ 5.91963948  4.55120748]
 [11.19192446  8.22634354]]
Iter 1: distances: 14118.543942631686
Iter 2: distances: 725.9983155026362
Iter 3: distances: 225.47101009437293
Iter 4: distances: 88.37141900221106
Iter 5: distances: 67.881469735

Iter 8: distances: 516.9584628932705
Iter 9: distances: 467.3516973268618
Iter 10: distances: 366.542186348738
Iter 11: distances: 303.7587807908316
Iter 12: distances: 273.74722072281054
Iter 13: distances: 229.1156400821021
Iter 14: distances: 159.41918663126438
Iter 15: distances: 129.19620335099788
Iter 16: distances: 106.29449711601578
Iter 17: distances: 106.399743389833
Iter 18: distances: 112.12257759330976
Iter 19: distances: 87.57343549137323
Iter 20: distances: 68.58407532772142
Iter 21: distances: 57.247370268148806
Iter 22: distances: 42.62387262446186
Iter 23: distances: 44.53239852629939
Iter 24: distances: 54.18467964181236
Iter 25: distances: 48.456953733318116
Iter 26: distances: 47.07802028263416
Iter 27: distances: 39.804824063384366
Iter 28: distances: 30.003519212091412
Iter 29: distances: 29.744003053293994
Iter 30: distances: 33.08240669578698
Iter 31: distances: 33.09551043907365
Iter 32: distances: 47.91262984645017
Iter 33: distances: 43.540811378420884
Iter 